# Definition
<ol>
    <li> Objects that require a lot of ceremony to create. Example having an object with 10 initializer arguments. It's hard to develop and manage all this arguments </li>
    <li> In builder case we can call separated methods to build this large object, building piece by piece instead of build in one time a large object</li>
    <li>Builder provides an API for constructing an object step-by-step</li>
    <b><i><li>When piecewise object construction is complicated, provide an API for doing it succinctly</li></i></b>
</ol>

### Simple Cenario

In [2]:
text = 'hello'
parts = ['<p>', text, '</p>']
print(''.join(parts))

<p>hello</p>


In [3]:
words = ['hello', 'world']
parts = ['<ul>']
for w in words:
    parts.append(f' <li>{w}</li>')
parts.append('</ul>')
print('\n'.join(parts))

<ul>
 <li>hello</li>
 <li>world</li>
</ul>


In [19]:
class HtmlElement:
    indent_size = 2
    
    def __init__(self, name='', text=''):
        self.name= name
        self.text = text
        self.elements = []
        
        
    def __str(self, indent):
        lines = []
        i = ' ' * (indent * self.indent_size)
        lines.append(f'{i}<{self.name}>')
        
        if self.text:
            i1 = ' ' * ((indent + 1) * self.indent_size)
            lines.append(f'{i1}{self.text}')
            
        for e in self.elements:
            lines.append(e.__str(indent + 1))
            
        lines.append(f'{i}</{self.name}>')
        return '\n'.join(lines)
    
    
    def __str__(self):
        return self.__str(0) 
    
    @staticmethod
    def create(name):
        return HtmlBuilder(name)

In [20]:
class HtmlBuilder:
    
    def __init__(self, root_name):
        self.root_name = root_name
        self.__root = HtmlElement(name=root_name)
        
    def add_child(self, child_name, child_text):
        self.__root.elements.append(
            HtmlElement(child_name, child_text)
        )
        
    def add_child_fluent(self, child_name, child_text):
        self.__root.elements.append(
            HtmlElement(child_name, child_text)
        )
        # add the return self allows you to chain the invocations one after another
        # THIS
        # builder.add_child('li', 'hello').add_child('li', 'world')
        # INSTEAD OF THIS
        # builder.add_child('li', 'hello')
        # builder.add_child('li', 'world')
        return self
        
    def __str__(self):
        return str(self.__root)

In [21]:
builder = HtmlBuilder('ul')
builder.add_child('li', 'hello')
builder.add_child('li', 'world')
print('Ordinary builder')
print(builder)

Ordinary builder
<ul>
  <li>
    hello
  </li>
  <li>
    world
  </li>
</ul>


In [22]:
builder = HtmlBuilder('ul')
builder.add_child_fluent('li', 'hello').\
        add_child_fluent('li', 'world')
print('Ordinary builder')
print(builder)

Ordinary builder
<ul>
  <li>
    hello
  </li>
  <li>
    world
  </li>
</ul>


In [24]:
builder = HtmlElement.create('ul')
builder.add_child_fluent('li', 'hello').\
        add_child_fluent('li', 'world')
print('Ordinary builder')
print(builder)

Ordinary builder
<ul>
  <li>
    hello
  </li>
  <li>
    world
  </li>
</ul>


# Builder Facets

In [26]:
class Person:
    def __init__(self):
        #address
        self.street_addres = None
        self.postcode = None
        self.city = None
        
        #employment
        self.company_name = None
        self.position = None
        self.annual_income = None
        
    def __str__(self) -> str:
        return f'Address: {self.street_address}, {self.postcode}, {self.city}' +\
                f'Employed at {self.company_name} as {self.postcode} earning {self.annual_income}'

In [29]:
class PersonBuilder:
    def __init__(self, person=Person()):
        self.person = person
        
    @property
    def works(self):
        return PersonJobBuilder(self.person)
    
    @property
    def lives(self):
        return PersonAddressBuilder(self.person)
    
    def build(self):
        return self.person
    
class PersonJobBuilder(PersonBuilder):
    def __init__(self, person):
        super().__init__(person)
        
    def at(self,company_name):
        self.person.company_name = company_name
        return self
    
    def as_a(self,position):
        self.person.position = position
        return self
    
    def earning(self,annual_income):
        self.person.annual_income = annual_income
        return self
    
    
    
class PersonAddressBuilder(PersonBuilder):
    def __init__(self, person):
        super().__init__(person)
        
    def at(self,street_address):
        self.person.street_address = street_address
        return self
    
    def with_postcode(self,postcode):
        self.person.postcode = postcode
        return self
    
    def in_city(self,city):
        self.person.city = city
        return self

In [31]:
pb = PersonBuilder()
person = pb\
    .lives\
        .at('123 Lodon Road')\
        .in_city('London')\
        .with_postcode('SOAKS')\
    .works\
        .at('Fabrikam')\
        .as_a('engineer')\
        .earning(123000)\
    .build()
print(person)
        
    

Address: 123 Lodon Road, SOAKS, LondonEmployed at Fabrikam as SOAKS earning 123000


# Builder Inheritance

In [33]:
class Person:
    def __init__(self):
        self.name = None
        self.position = None
        self.date_of_birth = None
        
    def __str__(self):
        return f'{self.name} born on {self.date_of_birth} ' +\
                f'works as {self.position}'
    
    @staticmethod
    def new():
        return PersonBuilder()
    
    
class PersonBuilder:
    def __init__(self):
        self.person = Person()
        
    def build(self):
        return self.person
    
    
class PersonInfoBuilder(PersonBuilder):
    def called(self, name):
        self.person.name = name
        return self
    
class PersonJobBuilder(PersonInfoBuilder):
    def work_as_a(self,position):
        self.person.position = position
        return self
    
    
class PersonBirthDateBuilder(PersonJobBuilder):
    def born(self, date_of_birth):
        self.person.date_of_birth = date_of_birth
        return self

In [35]:
pb = PersonBirthDateBuilder()
me = pb\
        .called('Dmitri')\
        .work_as_a('Quant')\
        .born('1/1/1980')\
        .build()
print(me)

Dmitri born on 1/1/1980 works as Quant
